https://www.kobis.or.kr/kobis/business/main/main.do

In [1]:
import requests
from bs4 import BeautifulSoup
import time
from tqdm.notebook import tqdm
from urllib.parse import quote

import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from datetime import datetime, timedelta

In [2]:
best_movie = pd.read_excel("[KOBIS] 역대_박스오피스(2023.05.11).xlsx", header=4, engine="openpyxl")
best_movie['영화명']

0               명량
1             극한직업
2        신과함께-죄와 벌
3             국제시장
4       어벤져스: 엔드게임
          ...     
195    해리 포터와 불의 잔
196         비긴 어게인
197             태풍
198             킹콩
199         부러진 화살
Name: 영화명, Length: 200, dtype: object

In [241]:
movie_name = best_movie['영화명'][0]
# movie_name = '스즈메의 문단속'
keyword = "영화 "+ str(movie_name) +" 평점"
keyword

'영화 명량 평점'

In [90]:
# 셀레니움으로 네이버 창 열기
driver = webdriver.Chrome()
time.sleep(2)
url = "https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&query=%EC%98%81%ED%99%94+%EB%AA%85%EB%9F%89+%ED%8F%89%EC%A0%90&oquery=%EC%98%81%ED%99%94+%EB%AA%85%EB%9F%89+%EC%A0%95%EB%B3%B4&tqi=iaz3Ysp0YiRssLJ4UA4ssssss8Z-510880"
driver.get(url)
time.sleep(3)

In [15]:
# movie_name = '부러진 화살'
movie_name = '극한직업'
keyword = "영화 "+ str(movie_name) +" 평점"
keyword

'영화 극한직업 평점'

In [91]:
# 검색어 입력 및 검색
driver.find_element(By.CSS_SELECTOR,'div.greenbox input#nx_query').clear()
driver.find_element(By.CSS_SELECTOR,'div.greenbox input#nx_query').send_keys(keyword)
driver.find_element(By.CSS_SELECTOR,'button.bt_search').click()
time.sleep(2)

In [21]:
# 공감순 선택
driver.find_element(By.CSS_SELECTOR, 'li.tab._sorting_select_order._trigger span.ico_check_wrap').click()
time.sleep(1)
driver.find_element(By.CSS_SELECTOR, 'div.cm_filter_area._root._sorting_select_order ul li:nth-child(1) a').click()
time.sleep(1)

In [201]:
# 리뷰 스크롤 내리기
driver.find_element(By.CSS_SELECTOR,'div.lego_review_list._scroller').click()

# 10번
for i in range(0,3):
    driver.find_element(By.XPATH,'/html/body').send_keys(Keys.END)
    time.sleep(0.5)

In [22]:
soup = BeautifulSoup(driver.page_source, "html.parser")

In [180]:
제목 = soup.select_one('div._au_movie_content_wrap span strong._text').text
제목

'부러진 화살'

In [181]:
전체 = soup.select_one('span.area_star_number').text
참여자수 = soup.select_one('span.area_people').text.split('명 참여')[0]
남자 = soup.select_one('div.area_card_male span.area_star_number').text
여자 = soup.select_one('div.area_card_female span.area_star_number').text
print(전체, 참여자수, 남자, 여자)

8.91 7,378 8.68 9.30


In [182]:
_10대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(1) span.this_text_num').text
_20대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(2) span.this_text_num').text
_30대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(3) span.this_text_num').text
_40대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(4) span.this_text_num').text
_50대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(5) span.this_text_num').text
print(_10대, _20대, _30대, _40대, _50대)

9.18 8.77 8.88 9.01 8.55


In [280]:
lis = soup.select('div.lego_review_list._scroller ul._item_wrapper li')
li= lis[0]
관람객별점 = li.select_one('div.area_text_box').text.split('별점(10점 만점 중)')[1]
관람객리뷰 = li.select_one('div.area_text_expand span.desc._text').text.strip()
print(관람객별점, 관람객리뷰)

10 가볍게 생각 없이 즐길수 있는 영화 잘봤습니다


In [196]:
# 평론가 클릭 및 스크롤 내리기
driver.find_element(By.CSS_SELECTOR,'div.cm_tap_area._tab_wrap ul li:nth-child(2) a span').click()
driver.find_element(By.CSS_SELECTOR,'div.lego_critic_outer._scroller').click()

for i in range(0,3):
    driver.find_element(By.XPATH,'/html/body').send_keys(Keys.END)
    time.sleep(0.5)

In [85]:
lis = soup.select('div.lego_critic_outer._scroller ul.area_ulist li')
li= lis[0]
평론가별점 = li.select_one('div.area_text_box').text.split('별점(10점 만점 중)')[1]
평론가리뷰 = li.select_one('div.area_text_expand').text.strip()
print(평론가별점, 평론가리뷰)

7.0 해체 위기에 놓인 마약 수사반이 잠복근무를 위해 치킨집을 인수한다. 공무원이 자영업자가 되면... 펼쳐보기


In [ ]:
#main_pack > div.sc_new.cs_common_module.case_empasis.color_5._au_movie_content_wrap > div.cm_content_wrap > div.cm_content_area._cm_content_area_rating > div > div:nth-child(3) > div > ul > li:nth-child(6) > div.area_text_expand > span

In [116]:
for num in range(1,len(driver.find_elements(By.CSS_SELECTOR,'div.lego_critic_outer._scroller ul.area_ulist li'))+1):
    try: 
        driver.find_element(By.CSS_SELECTOR,f'div.lego_critic_outer._scroller ul.area_ulist li:nth-child({num}) div.area_text_expand button').click()
        time.sleep(1)
    except:
        pass

In [74]:
# 기본정보 클릭 러닝타임 추출
driver.find_element(By.CSS_SELECTOR,'div.sub_tap_area._scrolling_wrapper_common_tab._scroll_mover ul li:nth-child(2)').click()
time.sleep(2)

divs = driver.find_elements(By.CSS_SELECTOR,'div.cm_content_area._cm_content_area_info div.detail_info dl div')

for div in divs:
    if '러닝타임' in div.text:
        러닝타임 = div.text.split()[1]
    elif '개봉' in div.text:
        개봉 = datetime.strptime(div.text.split()[1], '%Y.%m.%d.')
print(러닝타임, 개봉)

128분 2014-07-30 00:00:00


# 영화정보, 평점, 리뷰 크롤링

In [117]:
# 역대 영화 200개 데이터 불러오기
best_movie = pd.read_excel("[KOBIS] 역대_박스오피스(2023.05.11).xlsx", header=4, engine="openpyxl")

# 빈 데이터프레임 만들기
df1 = pd.DataFrame(columns = ['순위', '영화명', '개봉일', '매출액', '관객수', '스크린수', '상영횟수', '대표국적', '국적', '배급사', 
                              '제목', '러닝타임', '개봉', '전체', '참여자수', '남자', '여자', '_10대', '_20대', '_30대', '_40대', '_50대'])
df7 = pd.DataFrame(columns = ['구분','별점','리뷰','제목','개봉','영화명','개봉일'])

# 실패시 담을 리스트 만들기
fail_list=[]

# 셀레니움으로 네이버 영화 검색 창 열기
driver = webdriver.Chrome()
time.sleep(2)
url = "https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&query=%EC%98%81%ED%99%94+%EB%AA%85%EB%9F%89+%ED%8F%89%EC%A0%90&oquery=%EC%98%81%ED%99%94+%EB%AA%85%EB%9F%89+%ED%8F%89%EC%A0%90&tqi=iaXngsp0Jywssd6FGoKssssstH0-119566"
driver.get(url)
time.sleep(3)

### 크롤링 ###

for i in tqdm(range(0,len(best_movie['영화명']))):
# for i in tqdm(range(0,2)):

    # 역대 파일 i번째 인덱스의 기본 정보 추출 (시리즈형태와 데이터프레임 형태)
    info = best_movie.iloc[i]
    info_df = best_movie.iloc[i:i+1]
    
    try: 
        # 검색어 설정
        keyword = "영화 "+ str(info['영화명']) +" 평점"

        # 검색어 입력 및 검색
        driver.find_element(By.CSS_SELECTOR,'div.greenbox input#nx_query').clear()
        driver.find_element(By.CSS_SELECTOR,'div.greenbox input#nx_query').send_keys(keyword)
        driver.find_element(By.CSS_SELECTOR,'button.bt_search').click()
        time.sleep(3)

        # 리뷰 공감순 선택
        driver.find_element(By.CSS_SELECTOR, 'li.tab._sorting_select_order._trigger span.ico_check_wrap').click()
        time.sleep(2)
        driver.find_element(By.CSS_SELECTOR, 'div.cm_filter_area._root._sorting_select_order ul li:nth-child(1) a').click()
        time.sleep(2)

        # 관람객 리뷰 스크롤 내리기
        driver.find_element(By.CSS_SELECTOR,'div.lego_review_list._scroller').click()
        time.sleep(1)

        for j in range(0,2):
            driver.find_element(By.XPATH,'/html/body').send_keys(Keys.END)
            time.sleep(1)

        # 평론가 클릭 및 스크롤 내리기
        driver.find_element(By.CSS_SELECTOR,'div.cm_tap_area._tab_wrap ul li:nth-child(2) a span').click()
        driver.find_element(By.CSS_SELECTOR,'div.lego_critic_outer._scroller').click()
        time.sleep(1)

        for k in range(0,2):
            driver.find_element(By.XPATH,'/html/body').send_keys(Keys.END)
            time.sleep(1)
        
        # 평론가 리뷰 중 가려진 텍스트 펼치기
        for num in range(1,len(driver.find_elements(By.CSS_SELECTOR,'div.lego_critic_outer._scroller ul.area_ulist li'))+1):
            try: 
                driver.find_element(By.CSS_SELECTOR,f'div.lego_critic_outer._scroller ul.area_ulist li:nth-child({num}) div.area_text_expand button').click()
                time.sleep(1)
            except:
                pass 
        
        # soup에 html 담기
        soup = BeautifulSoup(driver.page_source, "html.parser")
        
        # 변수 초기화
        제목 = "-"
        전체 = "-"
        참여자수 = "-"
        남자 = "-"
        여자 = "-"

        _10대 = "-"
        _20대 = "-"
        _30대 = "-"
        _40대 = "-"
        _50대 = "-"

        
        # 변수에 해당되는 값 추출해서 담기
        제목 = soup.select_one('div._au_movie_content_wrap span strong._text').text
        전체 = soup.select_one('span.area_star_number').text
        참여자수 = soup.select_one('span.area_people').text.split('명 참여')[0]
        남자 = soup.select_one('div.area_card_male span.area_star_number').text
        여자 = soup.select_one('div.area_card_female span.area_star_number').text

        _10대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(1) span.this_text_num').text
        _20대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(2) span.this_text_num').text
        _30대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(3) span.this_text_num').text
        _40대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(4) span.this_text_num').text
        _50대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(5) span.this_text_num').text

        # 빈 리스트 만들기
        df4 = []
        df5 = []

        # 관람객 리뷰
        lis1 = soup.select('div.lego_review_list._scroller ul._item_wrapper li')
        
        관람객별점 = '-'
        관람객리뷰 = '-'
        
        for li1 in lis1:
            관람객별점 = li1.select_one('div.area_text_box').text.split('별점(10점 만점 중)')[1]
            관람객리뷰 = li1.select_one('div.area_text_expand span.desc._text').text.strip()
            df4.append(['관람객', 관람객별점, 관람객리뷰])

        # 평론가 리뷰
        lis2 = soup.select('div.lego_critic_outer._scroller ul.area_ulist li')
        
        평론가별점 = '-'
        평론가리뷰 = '-'
        
        for li2 in lis2:
            평론가별점 = li2.select_one('div.area_text_box').text.split('별점(10점 만점 중)')[1]
            평론가리뷰 = li2.select_one('div.area_text_expand').text.strip()
            df5.append(['평론가', 평론가별점, 평론가리뷰])

        # 기본정보 클릭 개봉일, 러닝타임 추출
        driver.find_element(By.CSS_SELECTOR,'div.sub_tap_area._scrolling_wrapper_common_tab._scroll_mover ul li:nth-child(2)').click()
        time.sleep(3)

        sub_soup = BeautifulSoup(driver.page_source, "html.parser")
    
        divs = sub_soup.select('div.cm_content_area._cm_content_area_info div.detail_info dl div')
        
        러닝타임 = '-'
        개봉 = '-'

        for div in divs:
            if '러닝타임' in div.text:
                러닝타임 = div.text.split()[1]
            elif '개봉' in div.text:
                개봉 = datetime.strptime(div.text.split()[1], '%Y.%m.%d.')

         
        # [평점]
        # 역대 기본 정보와 크롤링한 정보 합치기
        df2_list = [[제목, 러닝타임, 개봉, 전체, 참여자수, 남자, 여자, _10대, _20대, _30대, _40대, _50대]]
        df2 = pd.DataFrame(df2_list, columns = ['제목', '러닝타임', '개봉', '전체', '참여자수', '남자', '여자', '_10대', '_20대', '_30대', '_40대', '_50대'], index=[i])
        df3 = pd.concat([info_df, df2], axis=1)
        
        # 최종 데이터프레임에 추가하기
        df1 = pd.concat([df1, df3])
        
        # [리뷰]
        # 역대 기본 정보 영화명, 개봉일과 크롤링한 정보 합치기
        df4 = pd.DataFrame(df4, columns = ['구분','별점','리뷰'])
        df4['제목'] = 제목
        df4['개봉'] = 개봉
        df4['영화명'] = info['영화명']
        df4['개봉일'] = info['개봉일']

        df5 = pd.DataFrame(df5, columns = ['구분','별점','리뷰'])
        df5['제목'] = 제목
        df5['개봉'] = 개봉
        df5['영화명'] = info['영화명']
        df5['개봉일'] = info['개봉일']
         
        df6 = pd.concat([df4,df5])
        df7 = pd.concat([df7,df6])
        
    # 실패시 fail_list에 영화명 개봉일 담기 
    except:
        print(info['영화명'])
        fail_list.append([info['영화명'],info['개봉일']])
        
# 최종 데이터 프레임 컬럼명 설정
best_movie_star = pd.DataFrame(df1, columns=['순위', '영화명', '개봉일', '매출액', '관객수', '스크린수', '상영횟수', '대표국적', '국적', '배급사', 
                                             '제목', '러닝타임', '개봉', '전체', '참여자수', '남자', '여자', '_10대', '_20대', '_30대', '_40대', '_50대'])
best_movie_review = pd.DataFrame(df7, columns=['구분','별점','리뷰','제목','개봉','영화명','개봉일'])
best_movie_review = best_movie_review[['영화명','개봉일', '제목','개봉', '구분','별점','리뷰']]
fail_list_df = pd.DataFrame(fail_list, columns=['영화명','개봉일'])

# 엑셀 및 csv 파일로 저장
best_movie_star.to_excel('[크롤링] 역대_네이버_영화_평점_수정.xlsx', index=False)
best_movie_review.to_excel('[크롤링] 역대_네이버_영화_리뷰_수정.xlsx', index=False)
fail_list_df.to_csv('[크롤링] 역대_네이버_영화_실패_수정.csv', index=False)

  0%|          | 0/2 [00:00<?, ?it/s]

In [163]:
best_movie_star.to_csv('[크롤링] 역대_네이버_영화_평점_수정.csv', index=False)
best_movie_review.to_csv('[크롤링] 역대_네이버_영화_리뷰_수정.csv', index=False)

In [28]:
fail_list = pd.read_csv('[크롤링] 역대_네이버_영화_실패_수정.csv')
fail_list = fail_list.drop(index=0)
fail_list.to_excel('[크롤링] 역대_네이버_영화_실패_수정.xlsx', index=False)

# fail_list - URL 기준 크롤링

In [60]:
best_movie = pd.read_excel("[KOBIS] 역대_박스오피스(2023.05.11).xlsx", header=4, engine="openpyxl")

df1 = pd.DataFrame(columns = ['순위', '영화명', '개봉일', '매출액', '관객수', '스크린수', '상영횟수', '대표국적', '국적', '배급사', 
                              '제목', '러닝타임', '개봉', '전체', '참여자수', '남자', '여자', '_10대', '_20대', '_30대', '_40대', '_50대'])
df7 = pd.DataFrame(columns = ['구분','별점','리뷰','제목','개봉','영화명','개봉일'])

fail_list=[]

# 셀레니움으로 네이버 창 열기
driver = webdriver.Chrome()
time.sleep(2)
url = "https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&query=%EC%98%81%ED%99%94+%EB%AA%85%EB%9F%89+%ED%8F%89%EC%A0%90&oquery=%EC%98%81%ED%99%94+%EB%AA%85%EB%9F%89+%ED%8F%89%EC%A0%90&tqi=iaXngsp0Jywssd6FGoKssssstH0-119566"
driver.get(url)
time.sleep(3)

# 실패 리스트에서 URL 추가한 파일 불러오기
fail_list_link = pd.read_excel('[크롤링] 역대_네이버_영화_실패_수정.xlsx')
fail_list_link

for fail_index in tqdm(range(0,len(fail_list_link['영화명']))):
    
    movie_keyword = fail_list_link['영화명'][fail_index]

    i = int(best_movie[best_movie['영화명']==movie_keyword]['순위'])- 1
    info = best_movie.iloc[i]
    info_df = best_movie.iloc[i:i+1]

    try: 
        # 검색어 설정
        keyword = "영화 "+ str(movie_keyword) +" 평점"

        # 해당 영화 정보 링크 들어가기
        sub_url = fail_list_link['링크'][fail_index]
        driver.get(sub_url)
        time.sleep(3)

        # 리뷰 공감순 선택
        driver.find_element(By.CSS_SELECTOR, 'li.tab._sorting_select_order._trigger span.ico_check_wrap').click()
        time.sleep(2)
        driver.find_element(By.CSS_SELECTOR, 'div.cm_filter_area._root._sorting_select_order ul li:nth-child(1) a').click()
        time.sleep(2)

        # 관람객 리뷰 스크롤 내리기
        driver.find_element(By.CSS_SELECTOR,'div.lego_review_list._scroller').click()
        time.sleep(1)

        for j in range(0,2):
            driver.find_element(By.XPATH,'/html/body').send_keys(Keys.END)
            time.sleep(1)

        # 평론가 클릭 및 스크롤 내리기
        driver.find_element(By.CSS_SELECTOR,'div.cm_tap_area._tab_wrap ul li:nth-child(2) a span').click()
        driver.find_element(By.CSS_SELECTOR,'div.lego_critic_outer._scroller').click()
        time.sleep(1)

        for k in range(0,2):
            driver.find_element(By.XPATH,'/html/body').send_keys(Keys.END)
            time.sleep(1)
        
        # 평론가 리뷰 중 가려진 텍스트 펼치기
        for num in range(1,len(driver.find_elements(By.CSS_SELECTOR,'div.lego_critic_outer._scroller ul.area_ulist li'))+1):
            try: 
                driver.find_element(By.CSS_SELECTOR,f'div.lego_critic_outer._scroller ul.area_ulist li:nth-child({num}) div.area_text_expand button').click()
                time.sleep(1)
            except:
                pass 
        
        # soup에 html 담기
        soup = BeautifulSoup(driver.page_source, "html.parser")
        
        # 변수 초기화
        제목 = "-"
        전체 = "-"
        참여자수 = "-"
        남자 = "-"
        여자 = "-"

        _10대 = "-"
        _20대 = "-"
        _30대 = "-"
        _40대 = "-"
        _50대 = "-"

        
        # 변수에 해당되는 값 추출해서 담기
        제목 = soup.select_one('div._au_movie_content_wrap span strong._text').text
        전체 = soup.select_one('span.area_star_number').text
        참여자수 = soup.select_one('span.area_people').text.split('명 참여')[0]
        남자 = soup.select_one('div.area_card_male span.area_star_number').text
        여자 = soup.select_one('div.area_card_female span.area_star_number').text

        _10대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(1) span.this_text_num').text
        _20대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(2) span.this_text_num').text
        _30대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(3) span.this_text_num').text
        _40대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(4) span.this_text_num').text
        _50대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(5) span.this_text_num').text

        # 빈 리스트 만들기
        df4 = []
        df5 = []

        # 관람객 리뷰
        lis1 = soup.select('div.lego_review_list._scroller ul._item_wrapper li')
        
        관람객별점 = '-'
        관람객리뷰 = '-'
        
        for li1 in lis1:
            관람객별점 = li1.select_one('div.area_text_box').text.split('별점(10점 만점 중)')[1]
            관람객리뷰 = li1.select_one('div.area_text_expand span.desc._text').text.strip()
            df4.append(['관람객', 관람객별점, 관람객리뷰])

        # 평론가 리뷰
        lis2 = soup.select('div.lego_critic_outer._scroller ul.area_ulist li')
        
        평론가별점 = '-'
        평론가리뷰 = '-'
        
        for li2 in lis2:
            평론가별점 = li2.select_one('div.area_text_box').text.split('별점(10점 만점 중)')[1]
            평론가리뷰 = li2.select_one('div.area_text_expand').text.strip()
            df5.append(['평론가', 평론가별점, 평론가리뷰])

        # 기본정보 클릭 개봉일, 러닝타임 추출
        driver.find_element(By.CSS_SELECTOR,'div.sub_tap_area._scrolling_wrapper_common_tab._scroll_mover ul li:nth-child(2)').click()
        time.sleep(3)

        sub_soup = BeautifulSoup(driver.page_source, "html.parser")
    
        divs = sub_soup.select('div.cm_content_area._cm_content_area_info div.detail_info dl div')
        
        러닝타임 = '-'
        개봉 = '-'

        for div in divs:
            if '러닝타임' in div.text:
                러닝타임 = div.text.split()[1]
            elif '개봉' in div.text:
                개봉 = datetime.strptime(div.text.split()[1], '%Y.%m.%d.')

         
        # [평점]
        # 역대 기본 정보와 크롤링한 정보 합치기
        df2_list = [[제목, 러닝타임, 개봉, 전체, 참여자수, 남자, 여자, _10대, _20대, _30대, _40대, _50대]]
        df2 = pd.DataFrame(df2_list, columns = ['제목', '러닝타임', '개봉', '전체', '참여자수', '남자', '여자', '_10대', '_20대', '_30대', '_40대', '_50대'], index=[i])
        df3 = pd.concat([info_df, df2], axis=1)
        
        # 최종 데이터프레임에 추가하기
        df1 = pd.concat([df1, df3])
        
        # [리뷰]
        # 역대 기본 정보 영화명, 개봉일과 크롤링한 정보 합치기
        df4 = pd.DataFrame(df4, columns = ['구분','별점','리뷰'])
        df4['제목'] = 제목
        df4['개봉'] = 개봉
        df4['영화명'] = info['영화명']
        df4['개봉일'] = info['개봉일']

        df5 = pd.DataFrame(df5, columns = ['구분','별점','리뷰'])
        df5['제목'] = 제목
        df5['개봉'] = 개봉
        df5['영화명'] = info['영화명']
        df5['개봉일'] = info['개봉일']
         
        df6 = pd.concat([df4,df5])
        df7 = pd.concat([df7,df6])
        
    # 실패시 fail_list에 영화명 개봉일 담기 
    except:
        print(info['영화명'])
        fail_list.append([info['영화명'],info['개봉일']])
        
# 최종 데이터 프레임 컬럼명 설정
best_movie_star = pd.DataFrame(df1, columns=['순위', '영화명', '개봉일', '매출액', '관객수', '스크린수', '상영횟수', '대표국적', '국적', '배급사', 
                                             '제목', '러닝타임', '개봉', '전체', '참여자수', '남자', '여자', '_10대', '_20대', '_30대', '_40대', '_50대'])
best_movie_review = pd.DataFrame(df7, columns=['구분','별점','리뷰','제목','개봉','영화명','개봉일'])
best_movie_review = best_movie_review[['영화명','개봉일', '제목','개봉', '구분','별점','리뷰']]
fail_list_df = pd.DataFrame(fail_list, columns=['영화명','개봉일'])

  0%|          | 0/36 [00:00<?, ?it/s]

In [61]:
best_movie_star_df = pd.read_excel('[크롤링] 역대_네이버_영화_평점_수정.xlsx')
best_movie_review_df = pd.read_excel('[크롤링] 역대_네이버_영화_리뷰_수정.xlsx')

In [69]:
best_movie_star_df = pd.concat([best_movie_star_df, best_movie_star])
best_movie_review_df = pd.concat([best_movie_review_df, best_movie_review])

In [70]:
best_movie_star_df.to_excel('[크롤링] 역대_네이버_영화_평점_수정1.xlsx', index=False)
best_movie_review_df.to_excel('[크롤링] 역대_네이버_영화_리뷰_수정1.xlsx', index=False)

In [67]:
best_movie_review[best_movie_review["개봉일"]!=best_movie_review['개봉']]

,영화명,개봉일,제목,개봉,구분,별점,리뷰


In [95]:
df = best_movie_review_df[best_movie_review_df['구분']=="관람객"]['영화명'].value_counts().to_frame()

In [100]:
df[df['영화명']<30]

,영화명
다크 나이트 라이즈,16
어벤져스,10
아바타,10


# 영화명 하나씩 가져오기

In [40]:
best_movie = pd.read_excel("[KOBIS] 역대_박스오피스(2023.05.11).xlsx", header=4, engine="openpyxl")

df1 = pd.DataFrame(columns = ['순위', '영화명', '개봉일', '매출액', '관객수', '스크린수', '상영횟수', '대표국적', '국적', '배급사', 
                              '제목', '러닝타임', '개봉', '전체', '참여자수', '남자', '여자', '_10대', '_20대', '_30대', '_40대', '_50대'])
df7 = pd.DataFrame(columns = ['구분','별점','리뷰','제목','개봉','영화명','개봉일'])

fail_list=[]

# 셀레니움으로 네이버 창 열기
driver = webdriver.Chrome()
time.sleep(2)
url = "https://search.naver.com/search.naver?sm=tab_hty.top&where=nexearch&query=%EC%98%81%ED%99%94+%EB%AA%85%EB%9F%89+%ED%8F%89%EC%A0%90&oquery=%EC%98%81%ED%99%94+%EB%AA%85%EB%9F%89+%ED%8F%89%EC%A0%90&tqi=iaXngsp0Jywssd6FGoKssssstH0-119566"
driver.get(url)
time.sleep(3)

fail_list_link = pd.read_excel('[크롤링] 역대_네이버_영화_실패_수정.xlsx')

movie_keyword = '아바타'

i = int(best_movie[best_movie['영화명']==movie_keyword]['순위'])- 1
info = best_movie.iloc[i]
info_df = best_movie.iloc[i:i+1]

try: 
    # 검색어 설정
    keyword = "영화 "+ str(movie_keyword) +" 평점"

    # 검색어 입력 및 검색
    sub_url = fail_list_link[fail_list_link['영화명']==movie_keyword].iloc[0]["링크"]
    driver.get(sub_url)
    time.sleep(3)

    # 리뷰 공감순 선택
    driver.find_element(By.CSS_SELECTOR, 'li.tab._sorting_select_order._trigger span.ico_check_wrap').click()
    time.sleep(2)
    driver.find_element(By.CSS_SELECTOR, 'div.cm_filter_area._root._sorting_select_order ul li:nth-child(1) a').click()
    time.sleep(2)

    # 관람객 리뷰 스크롤 내리기
    driver.find_element(By.CSS_SELECTOR,'div.lego_review_list._scroller').click()
    time.sleep(1)

    for j in range(0,2):
        driver.find_element(By.XPATH,'/html/body').send_keys(Keys.END)
        time.sleep(1)

    # 평론가 클릭 및 스크롤 내리기
    driver.find_element(By.CSS_SELECTOR,'div.cm_tap_area._tab_wrap ul li:nth-child(2) a span').click()
    driver.find_element(By.CSS_SELECTOR,'div.lego_critic_outer._scroller').click()
    time.sleep(1)

    for k in range(0,2):
        driver.find_element(By.XPATH,'/html/body').send_keys(Keys.END)
        time.sleep(1)

    # 평론가 리뷰 중 가려진 텍스트 펼치기
    for num in range(1,len(driver.find_elements(By.CSS_SELECTOR,'div.lego_critic_outer._scroller ul.area_ulist li'))+1):
        try: 
            driver.find_element(By.CSS_SELECTOR,f'div.lego_critic_outer._scroller ul.area_ulist li:nth-child({num}) div.area_text_expand button').click()
            time.sleep(1)
        except:
            pass 

    # soup에 html 담기
    soup = BeautifulSoup(driver.page_source, "html.parser")

    # 변수 초기화
    제목 = "-"
    전체 = "-"
    참여자수 = "-"
    남자 = "-"
    여자 = "-"

    _10대 = "-"
    _20대 = "-"
    _30대 = "-"
    _40대 = "-"
    _50대 = "-"


    # 변수에 해당되는 값 추출해서 담기
    제목 = soup.select_one('div._au_movie_content_wrap span strong._text').text
    전체 = soup.select_one('span.area_star_number').text
    참여자수 = soup.select_one('span.area_people').text.split('명 참여')[0]
    남자 = soup.select_one('div.area_card_male span.area_star_number').text
    여자 = soup.select_one('div.area_card_female span.area_star_number').text

    _10대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(1) span.this_text_num').text
    _20대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(2) span.this_text_num').text
    _30대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(3) span.this_text_num').text
    _40대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(4) span.this_text_num').text
    _50대 = soup.select_one('div.lego_rating_box_age ul.area_graph_age li:nth-child(5) span.this_text_num').text

    # 빈 리스트 만들기
    df4 = []
    df5 = []

    # 관람객 리뷰
    lis1 = soup.select('div.lego_review_list._scroller ul._item_wrapper li')

    관람객별점 = '-'
    관람객리뷰 = '-'

    for li1 in lis1:
        관람객별점 = li1.select_one('div.area_text_box').text.split('별점(10점 만점 중)')[1]
        관람객리뷰 = li1.select_one('div.area_text_expand span.desc._text').text.strip()
        df4.append(['관람객', 관람객별점, 관람객리뷰])

    # 평론가 리뷰
    lis2 = soup.select('div.lego_critic_outer._scroller ul.area_ulist li')

    평론가별점 = '-'
    평론가리뷰 = '-'

    for li2 in lis2:
        평론가별점 = li2.select_one('div.area_text_box').text.split('별점(10점 만점 중)')[1]
        평론가리뷰 = li2.select_one('div.area_text_expand').text.strip()
        df5.append(['평론가', 평론가별점, 평론가리뷰])

    # 기본정보 클릭 개봉일, 러닝타임 추출
    driver.find_element(By.CSS_SELECTOR,'div.sub_tap_area._scrolling_wrapper_common_tab._scroll_mover ul li:nth-child(2)').click()
    time.sleep(3)

    sub_soup = BeautifulSoup(driver.page_source, "html.parser")

    divs = sub_soup.select('div.cm_content_area._cm_content_area_info div.detail_info dl div')

    러닝타임 = '-'
    개봉 = '-'

    for div in divs:
        if '러닝타임' in div.text:
            러닝타임 = div.text.split()[1]
        elif '개봉' in div.text:
            개봉 = datetime.strptime(div.text.split()[1], '%Y.%m.%d.')


    # [평점]
    # 역대 기본 정보와 크롤링한 정보 합치기
    df2_list = [[제목, 러닝타임, 개봉, 전체, 참여자수, 남자, 여자, _10대, _20대, _30대, _40대, _50대]]
    df2 = pd.DataFrame(df2_list, columns = ['제목', '러닝타임', '개봉', '전체', '참여자수', '남자', '여자', '_10대', '_20대', '_30대', '_40대', '_50대'], index=[i])
    df3 = pd.concat([info_df, df2], axis=1)

    # 최종 데이터프레임에 추가하기
    df1 = pd.concat([df1, df3])

    # [리뷰]
    # 역대 기본 정보 영화명, 개봉일과 크롤링한 정보 합치기
    df4 = pd.DataFrame(df4, columns = ['구분','별점','리뷰'])
    df4['제목'] = 제목
    df4['개봉'] = 개봉
    df4['영화명'] = info['영화명']
    df4['개봉일'] = info['개봉일']

    df5 = pd.DataFrame(df5, columns = ['구분','별점','리뷰'])
    df5['제목'] = 제목
    df5['개봉'] = 개봉
    df5['영화명'] = info['영화명']
    df5['개봉일'] = info['개봉일']

    df6 = pd.concat([df4,df5])
    df7 = pd.concat([df7,df6])

# 실패시 fail_list에 영화명 개봉일 담기 
except:
    print(info['영화명'])
    fail_list.append([info['영화명'],info['개봉일']])

# 최종 데이터 프레임 컬럼명 설정
best_movie_star = pd.DataFrame(df1, columns=['순위', '영화명', '개봉일', '매출액', '관객수', '스크린수', '상영횟수', '대표국적', '국적', '배급사', 
                                         '제목', '러닝타임', '개봉', '전체', '참여자수', '남자', '여자', '_10대', '_20대', '_30대', '_40대', '_50대'])
best_movie_review = pd.DataFrame(df7, columns=['구분','별점','리뷰','제목','개봉','영화명','개봉일'])
best_movie_review = best_movie_review[['영화명','개봉일', '제목','개봉', '구분','별점','리뷰']]
fail_list_df = pd.DataFrame(fail_list, columns=['영화명','개봉일'])

In [146]:
best_movie_star_df = pd.read_excel('[크롤링] 역대_네이버_영화_평점_수정2.xlsx')
best_movie_review_df = pd.read_excel('[크롤링] 역대_네이버_영화_리뷰_수정2.xlsx')

In [147]:
best_movie_star_df = pd.concat([best_movie_star_df, best_movie_star])
best_movie_review_df = pd.concat([best_movie_review_df, best_movie_review])

In [148]:
best_movie_star_df.to_excel('[크롤링] 역대_네이버_영화_평점_수정3.xlsx', index=False)
best_movie_review_df.to_excel('[크롤링] 역대_네이버_영화_리뷰_수정3.xlsx', index=False)

# 이상요인 판단

In [151]:
best_movie_star_df = pd.read_excel('[크롤링] 역대_네이버_영화_평점_최종_수정.xlsx')
best_movie_review_df = pd.read_excel('[크롤링] 역대_네이버_영화_리뷰_최종_수정.xlsx')

In [152]:
best_movie_review_df.groupby(['영화명', '구분'])['구분'].count().to_frame()

구분
영화명          구분     
1987         관람객  30
             평론가  10
2012         관람객  30
             평론가  10
7급 공무원       관람객  30
...               ..
혹성탈출: 반격의 서막 평론가  10
화려한 휴가       관람객  30
             평론가   8
히말라야         관람객  30
             평론가  10

[398 rows x 1 columns]

In [155]:
df = best_movie_review_df[best_movie_review_df['구분']=="관람객"]['영화명'].value_counts().to_frame()
df[df['영화명']<30]

,영화명
어벤져스,20
아바타,19
다크 나이트 라이즈,16


In [162]:
# 관람객 리뷰 별점
best_movie_review_df[best_movie_review_df['구분']=="관람객"]['별점'].value_counts().to_frame()

,별점
10.0,3548
9.0,695
8.0,555
1.0,374
7.0,237
6.0,198
5.0,116
4.0,97
2.0,78
3.0,67
